In [1]:
# import torch
import sys
import IPython
import random
import os

# custom
from datahandlers.MyDataloader import *
from models.HubertCaption import * # network
from Trainer import *
    
TEST_BATCH_SIZE = 5

USE_CUDA = torch.cuda.is_available() 
# device = torch.device('cuda:1' if USE_CUDA else 'cpu')

In [2]:
import torch

In [2]:
model_path = '/data/valerii/AudioCaption/data/Train_record/hubert_audiocaps1/best_model'
model = HubertCaption()
params = torch.load(model_path, map_location='cuda:0')
model.load_state_dict(params) 

mapping network: num_head = 8 num_layers = 8 prefix_vector_length = 60
GPT2 has been freezed
GPT header is trainable


<All keys matched successfully>

In [4]:
# table_num = 1 : Evaluation on Clotho
# table_num = 2 : Evaluation on AudioCaps

# setting_num = 1 : train dataset == test dataset
# setting_num = 2 : train dataset != test dataset
# setting_num = 3 : overall datasets(Clotho & AudioCaps) <- need to test by using compressed audio

table_num = 2
setting_num = 1

if setting_num == 3 :
    is_settingnum_3 = True
else : 
    is_settingnum_3 = False

#model = get_model_in_table(table_num, setting_num, device).eval()


device = 'cpu'
#to get inference from all models
# model_old = get_model_in_table(2, 2, device, weights_path='/data/valerii/AudioCaption/data/Train_record/const500k_pad2_tok35_fus/best_model').eval()
# model_asaccl = get_model_in_table(2, 2, device, weights_path = '/data/valerii/AudioCaption/data/Train_record/as_ac_cl_2/best_model').eval()
# model_bbc = get_model_in_table(2, 2, device, weights_path = '/data/valerii/AudioCaption/data/Train_record/as_ac_cl_bbc_const600k_3/best_model').eval()
# model_authors = get_model_in_table(1, 3, device).eval()
model_bbc = get_model_in_table(2, 2, device, weights_path = '/data/valerii/AudioCaption/data/Train_record/bbc_model2/best_model',
                           prefix_size_dict = {"temporal_prefix_size" : 60, "global_prefix_size" : 20}).eval()
model_hearts = get_model_in_table(2, 2, device, weights_path = '/data/valerii/AudioCaption/data/Train_record/bbc_model_hearts2/best_model',
                           prefix_size_dict = {"temporal_prefix_size" : 60, "global_prefix_size" : 20}).eval()

/data/valerii/anaconda3/envs/Prefix/lib/python3.9/site-packages/torchlibrosa/stft.py:686: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  self.melW = librosa.filters.mel(sr=sr, n_fft=n_fft, n_mels=n_mels,


use GPT2 Tokenizer


In [ ]:
torch.save(model_hearts, "hearts_model.pth")

In [8]:
heartsounds = MakeDataset(None, './data/WavCaps/'+ "heartsounds", 'test',None,16000,tokens_size = None,tokenizer_type = None)
hearts = MakeDataset(None, './data/WavCaps/'+ "hearts_kaggle", "test", None, 16000, tokens_size = None,tokenizer_type = None)

get test dataset from ./data/WavCaps/heartsounds...: 100%|██████████| 391/391 [00:00<00:00, 91388.85it/s]
get test dataset from ./data/WavCaps/hearts_kaggle...: 100%|██████████| 327/327 [00:00<00:00, 102613.90it/s]


In [4]:
caps_dataset = AudioCapsDataset(None, './data/AudioCaps', 'test', 26, tokens_size = 50,tokenizer_type = None)
clotho_dataset = ClothoDataset(None, './data/Clotho', 'test', 26, tokens_size = 50,tokenizer_type = None)
bbc_sounds = MakeDataset(None, './data/WavCaps/'+ "bbc_sounds", "test", None, 16000, tokens_size = None,tokenizer_type = None)
soundbible = MakeDataset(None, './data/WavCaps/'+ "soundbible", "test", None, 16000, tokens_size = None,tokenizer_type = None)

get dataset...:  18%|█▊        | 177/960 [00:00<00:00, 1760.74it/s]

get dataset...: 100%|██████████| 1045/1045 [00:01<00:00, 643.56it/s]
get test dataset from ./data/WavCaps/bbc_sounds...: 100%|██████████| 31201/31201 [00:00<00:00, 190566.06it/s]
get test dataset from ./data/WavCaps/soundbible...: 100%|██████████| 1232/1232 [00:00<00:00, 70814.19it/s]


In [42]:
dataset = hearts
with torch.no_grad():
    #choosing file from test randomly
    audio_file, caption, path = random.choice(dataset)
    audio_file = audio_file.unsqueeze(0).to(device)
    # pred_caption_authors = model_authors(audio_file, None, beam_search = True)[0][0]
    # pred_caption_bbc = model_bbc(audio_file, None, beam_search = True)[0][0]
    # pred_caption_audioset = model_asaccl(audio_file, None, beam_search = True)[0][0]
    best_caption = model_bbc(audio_file, None, beam_search = True)[0][0]
    # old_caption = model_old(audio_file, None, beam_search = True)[0][0]
    hearts_caption = model_hearts(audio_file, None, beam_search = True)[0][0]
    print(f"Ground truth caption: {caption}\n")
    # print("Caps+Clotho by authors: ",  pred_caption_authors)
    # print("Caps+Clotho without compressions: ",  old_caption)
    print("New bbc model: ",  best_caption)
    print("Hearts: ",  hearts_caption)
    # print("Caps+Clotho+Audioset: ",  pred_caption_audioset)
    # print("BBC model predict: ",  pred_caption_bbc)
IPython.display.Audio(path, rate=16000)

Ground truth caption: normal heart sounds are present.

New bbc model:  the wind is blowing and heartbeats are heard.
Hearts:  the auscultatory findings indicated the presence of normal heart sounds.


: 

In [18]:
audio_file.shape

torch.Size([72660])

In [19]:
test_loader = MyDataLoader({"hearts_kaggle":True}, 'test', 5)

get test dataset from ./data/WavCaps/hearts_kaggle...: 100%|██████████| 327/327 [00:00<00:00, 67777.10it/s]


In [13]:
dataset = clotho_dataset
model.eval().to(0)
with torch.no_grad():
    audio_file, caption, path = random.choice(dataset)
    predict = model(audio_file.unsqueeze(0).to(0))[0][0]
    print(f"Ground truth caption: {caption}\n")
    print("Model predict:",  predict)
IPython.display.Audio(path, rate=16000)

Ground truth caption: an air vent turned on blowing the air around.

Model predict: a vehicle engine is idling.


: 

In [ ]:
# with open('/data/valerii/Audio_LLM/data/WavCaps/bbc_sounds/download_list.txt', 'w') as file:
#     for i in range(5, 25):
#         if i < 10: 
#             c = '0' + str(i)
#         else:
#             c = str(i)
#         file.write('https://huggingface.co/datasets/cvssp/WavCaps/resolve/main/Zip_files/BBC_Sound_Effects/BBC_Sound_Effects.z' + c +'\n')


In [2]:
def prepare_audio_file(audio_file_path):
    # prepare audio input=========
    SAMPLE_RATE = 16000
    set_length = 30

    audio_file, _ = torchaudio.load(audio_file_path)
    # audio_file = audio_file.squeeze(0)
    # #?audio_file = audio_file.squeeze(0) ????
    # # slicing
    # if audio_file.shape[0] > (SAMPLE_RATE * set_length) :
    #     audio_file = audio_file[:SAMPLE_RATE * set_length]
    # # zero padding
    # if audio_file.shape[0] < (SAMPLE_RATE * set_length) :
    #     pad_len = (SAMPLE_RATE * set_length) - audio_file.shape[0]
    #     pad_val = torch.zeros(1, pad_len)
    #     audio_file = torch.cat((audio_file, pad_val), dim=1)
    # # prepare audio input=========

    return audio_file

In [53]:
# audio_file, _ = torchaudio.load("./AudioCaps/test/_9mgOkzm-xg.wav")
# pad_val = torch.zeros(1, pad_len)
# audio_file = torch.cat((audio_file, pad_val), dim=1)

In [1]:
from numba import cuda
# cuda.select_device(1)
# cuda.close()
# !nvidia-smi

In [8]:
with torch.no_grad():
    #choosing file from test randomly
    audio_file, caption, path = random.choice(clotho_dataset)
    audio_file = audio_file.unsqueeze(0).to(device)
    pred_caption_authors = model_authors(audio_file, None, beam_search = True)[0][0]
    pred_caption_best = model_const(audio_file, None, beam_search = True)[0][0]
    pred_caption_clotho = model_clotho(audio_file, None, beam_search = True)[0][0]

    print(f"Ground truth caption: {caption}\n")
    print("Clotho only trained by authors: ", pred_caption_clotho)
    print("Caps+Clotho by authors: ",  pred_caption_authors)
    print("Caps+Clotho trained best: ",  pred_caption_best)

IPython.display.Audio("./Clotho/clotho_audio_files/test/" + path)

Ground truth caption: many voices overlap indistinctly in background while two male voices speak loudly and alternately.

Clotho only trained by authors:  people are talking in the background as a man talking in the background.
Caps+Clotho by authors:  a man is speaking and a crowd of people are talking in the background.
Caps+Clotho trained best:  a man is talking to another man while another man is talking to another man.


: 